## Graph to Smile is a script designed to convert molecular smiles to graphs. These graphs, in turn, are the inputs of the GCN of the paper "Graph neural networks and molecular docking as two complementary approaches for virtual screening: a case study on Cruzain"

## **Mounting Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## **Clonning github repository**

In [2]:
# Set working directory 
!git clone https://github.com/lemyp-cadd/gcn-docking.git

Cloning into 'gcn-docking'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 230 (delta 67), reused 151 (delta 63), pack-reused 75
Receiving objects: 100% (230/230), 10.42 MiB | 25.47 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [3]:
! pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 68.3 MB/s eta 0:00:00


In [4]:
import numpy as np
import sys
from rdkit import Chem

In [5]:
def adj_k(adj, k):

    ret = adj
    for i in range(0, k-1):
        ret = np.dot(ret, adj)  

    return convertAdj(ret)

def convertAdj(adj):

    dim = len(adj)
    a = adj.flatten()
    b = np.zeros(dim*dim)
    c = (np.ones(dim*dim)-np.equal(a,b)).astype('float64')
    d = c.reshape((dim, dim))

    return d

def convertToGraph(smiles_list, k):
    adj = []
    adj_norm = []
    features = []
    potency = []	
    maxNumAtoms = 50
    for i in smiles_list:
        # Mol
        iMol = Chem.MolFromSmiles(i.split(',')[1].strip())
        print(i)
        #Adj
        iAdjTmp = Chem.rdmolops.GetAdjacencyMatrix(iMol)
        # Feature
        if( iAdjTmp.shape[0] <= maxNumAtoms):
            # Feature-preprocessing
            iFeature = np.zeros((maxNumAtoms, 58))
            iFeatureTmp = []
            for atom in iMol.GetAtoms():
                iFeatureTmp.append( atom_feature(atom) ) ### atom features only
            iFeature[0:len(iFeatureTmp), 0:58] = iFeatureTmp ### 0 padding for feature-set
            features.append(iFeature)

            # Adj-preprocessing
            iAdj = np.zeros((maxNumAtoms, maxNumAtoms))
            iAdj[0:len(iFeatureTmp), 0:len(iFeatureTmp)] = iAdjTmp + np.eye(len(iFeatureTmp))
            adj.append(adj_k(np.asarray(iAdj), k))

            # Activity
            potency.append(i.split(',')[2].strip())			
    features = np.asarray(features)
    potency = np.asarray(potency, dtype='float64')	

    return adj, features, potency
    
def atom_feature(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),
                                      ['C', 'N', 'O', 'S', 'F', 'H', 'Si', 'P', 'Cl', 'Br',
                                       'Li', 'Na', 'K', 'Mg', 'Ca', 'Fe', 'As', 'Al', 'I', 'B',
                                       'V', 'Tl', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn',
                                       'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'Mn', 'Cr', 'Pt', 'Hg', 'Pb']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5]) +
                    [atom.GetIsAromatic()])    # (40, 6, 5, 6, 1)

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    #print list((map(lambda s: x == s, allowable_set)))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


In [6]:
%cd gcn-docking/tests/smiles_to_graph/

/content/gcn-docking/tests/smiles_to_graph


## **SMILES to GRAPH**

In [8]:
folder = "AID1478_train" # Use AID1478_train or AID1478_test

dbName = folder
length = 10000   # num molecules in graph input files
k = 1        # neighbor distance

smiles_f = open(dbName+'/smiles.txt')
smiles_list = smiles_f.readlines()
print (len(smiles_list))
maxNum = int(len(smiles_list)/length)

for i in range(maxNum+1):
  lb = i*length
  ub = (i+1)*length
  adj, features, potency = convertToGraph(smiles_list[lb:ub], 1)
  print (np.asarray(features).shape)
  np.save(folder+'/adj/'+str(i)+'.npy', adj)
  np.save(folder+'/features/'+str(i)+'.npy', features)
  np.save(folder+'/potency/'+str(i)+'.npy', potency)


2025
16034213,CC1=CC=C(C=C1)C2=NN(C(=O)CC2)CC(=O)NCCCN(CC(C)C)CC(C)C,0

54681982,C[C@@]1(C2CC3C(C(=O)C(=C([C@]3(C(=O)C2=C(C4=C1C=CC=C4O)O)O)O)C(=O)N)N(C)C)O.Cl,0

16011720,CCN(CC)CCNC(=O)NC1=CC2=C(C=C1)SC3=CC=CC=C3C(=O)N2CC4=CC=CC=C4,0

5307820,CC1CCC2=NN(C=C2C1)CC(=O)NC3CCCC3,0

3242441,C1=CC(=C(C=C1Cl)F)NC(=O)C2=CC=C(O2)COC3=C(C(=C(C(=C3F)F)F)F)F,1

5307994,CC1CN(CCN1C2=CC=CC(=C2)C)C(=O)CS(=O)(=O)CC3=C(OC(=N3)C4=CC=CC=C4F)C,0

644515,CC1=CC=C(C=C1)OCC2=NN=C(N2C3=CC(=CC=C3)OC)SCC(=O)N,0

1482220,C1CN(NC1=O)C(=O)NC2=CC=CC=C2,0

3932264,COC1=CC=CC=C1NS(=O)(=O)C2=CC(=C(C=C2)Cl)NC(=O)CN3C(=O)C4=CC=CC=C4C3=O,0

135470038,CCCC(=O)NC1=NC2=C(CCCC2)C(=O)N1,0

16013088,CC1=CC=C(C=C1)CN(CC2=CC=CO2)C(=O)COC3=C(C=C(C=C3)Cl)Cl,0

2384869,CC1=CC=CC=C1NC2=NCCCS2,0

11957134,CCOC(=O)NC(C(Cl)(Cl)Cl)SC1=CC=CC=C1,0

653862,COC1=CC=C(C=C1)S(=O)(=O)N2C(=CC(=N2)OC(=O)C3=CC=CO3)N,1

618556,CS(=O)(=O)N1CCN(CC1)C2=NC(=CS2)C3=CC=CC=C3,0

2055089,C1=CC2=C3C(=C1)C(=O)N(C(=O)C3=CC=C2)C4=CC5=C(C=C4)N=CC=C5,1

44371

## **Cheking**

In [9]:
# folder is the same one used in the previous box
folder = "AID1478_train" # Use AID1478_train or AID1478_test
import os
archivo = "0.npy"

i = 0
for root, dirs, files in os.walk(folder):
    if archivo in files:
        i = i+1
if i == 3:
  print("graphs created successfully")
else:
  print("graphs not created correctly")

graphs created successfully
